In [ ]:
import torch
a=torch.tensor(11)
a.shape

In [ ]:
weight=torch.ones(16,768,16,16)
weight*torch.tensor(12)

In [ ]:
import torch
Wq=torch.randint(0,255,(64,32,3,3))
# with open ('TxT//ConvWeight.txt','a') as ff:
#     #Wq的维度：[OC,IC,K,K]
#     for OC in range(Wq.shape[0]):#遍历全部输出通道
#         for K_Row in range(Wq.shape[2]):#遍历行
#             for K_Col in range(Wq.shape[3]):#遍历列
#                 for IC in range(0,Wq.shape[1],8):#遍历通道
#                     ff.write('%02x'%(int(Wq[OC,IC:IC+7,K_Row,K_Col].item())&0xff))
#                     ff.write("\n")
#     ff.close()
for OC in range(Wq.shape[0]):#遍历全部输出通道
    for K_Row in range(Wq.shape[2]):#遍历行
        for K_Col in range(Wq.shape[3]):#遍历列
            for IC in range(0,Wq.shape[1],8):#遍历通道
                # print(list(Wq[OC,IC:IC+2,K_Row,K_Col]))
                Hex0=Wq[OC,IC,K_Row,K_Col].item()&0xff
                Hex1=Wq[OC,IC+1,K_Row,K_Col].item()&0xff
                Hex2=Wq[OC,IC+2,K_Row,K_Col].item()&0xff
                Hex3=Wq[OC,IC+3,K_Row,K_Col].item()&0xff
                Hex4=Wq[OC,IC+4,K_Row,K_Col].item()&0xff
                Hex5=Wq[OC,IC+5,K_Row,K_Col].item()&0xff
                Hex6=Wq[OC,IC+6,K_Row,K_Col].item()&0xff
                Hex7=Wq[OC,IC+7,K_Row,K_Col].item()&0xff
                print('%02x%02x%02x%02x%02x%02x%02x%02x'%(Hex7,Hex6,Hex5,Hex4,Hex3,Hex2,Hex1,Hex0))
                # ff.write("\n")    

In [ ]:
import os
filename = r'E:\Transformer\Transformer_Arithmatic\Transformer_Main\TxT\Weight.txt'

# 判断文件是否存在不存在则创建一个
if not os.path.isfile(filename):
     fd = open(filename, mode="w", encoding="utf-8")


In [ ]:
import torch.nn as nn
picture_in=torch.rand(1,3,224,224)

nn.conv(picture_in)

In [3]:
for i in range(0,32):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
